In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',100)

tiendas_SARIMAX=['Brooklyn','Midtown_Village','Roxbury']
tiendas_XGB_24_L30 = ['Back_Bay','Yorktown']
tiendas_XGB_SC_L30 = ['Harlem','Total Tiendas']
tiendas_XGB_SC_L365 = ['Greenwich_Village','Queen_Village','South_End','Tribeca']
tiendas_XGB = tiendas_XGB_24_L30 + tiendas_XGB_SC_L30 + tiendas_XGB_SC_L365
df_hp = pd.read_csv("hiperparametros XGBOOST.csv",delimiter=";")
df_eventos = pd.read_csv("daily_calendar_with_events.csv")
df_eventos = df_eventos.dropna(subset=['event'])
df_venart = pd.read_csv("ventas por articulo.csv")
df_venart['fecha'] = pd.to_datetime(df_venart['fecha'], format = '%Y-%m-%d')

### PREDICCIONES POR TOTAL

In [2]:
ventas_total = df_venart.groupby([df_venart['fecha'].dt.date])['cifra'].sum().fillna(0).reset_index()
ventas_total.rename(columns={'cifra': 'Total Tiendas'}, inplace=True)
ventas_total.columns.name = None

predicciones_TOTALES = pd.DataFrame()

rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_total['fecha'] = pd.to_datetime(ventas_total['fecha'], format = '%Y-%m-%d')
ventas_total = pd.merge(pd.DataFrame({'fecha': rango}), ventas_total, on='fecha', how='left')
ventas_total = ventas_total.fillna(0)
ventas_total.set_index('fecha', inplace=True)
ventas_total['evento'] = ventas_total.index.isin(df_eventos['date']) 
tienda = "Total Tiendas"
print(f"Tienda XGB: {tienda}")
df = ventas_total[[tienda, 'evento']].copy()
if tienda in tiendas_XGB_24_L30:
   df = df.loc['2014-04-24':'2016-04-24']
# Agregar más características relevantes
df['mes'] = df.index.month
df['dia_semana'] = df.index.weekday
    
# Agregar los lags de 7, 30 y 365 días
df['lag_1'] = df[tienda].shift(1)
df['lag_7'] = df[tienda].shift(7)
df['lag_30'] = df[tienda].shift(30)
if tienda in tiendas_XGB_SC_L365:
   df['lag_365'] = df[tienda].shift(365)
  
# Eliminar filas con valores nulos que surgen por los lags
df = df.dropna()

# Preparar los datos para XGBoost, incluyendo las nuevas características de lag
if tienda in tiendas_XGB_SC_L365:
   X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
else:
   X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         
y_train = df[tienda].values

ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
model.fit(X_train, y_train)

# Predecir los próximos 28 días después del conjunto de prueba
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
future_mes = future_dates.month
future_dia_semana = future_dates.weekday
    
# Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
future_lag_1 = df[tienda].shift(1).iloc[-28:].values
future_lag_7 = df[tienda].shift(7).iloc[-28:].values
future_lag_30 = df[tienda].shift(30).iloc[-28:].values
if tienda in tiendas_XGB_SC_L365:
   future_lag_365 = df[tienda].shift(365).iloc[-28:].values
   X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
else:
   X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

forecast_28_days = model.predict(X_future)

# Crear DataFrame con las predicciones
df_predicciones = pd.DataFrame({
   'Fecha': future_dates,
   'Prediccion_Ventas': forecast_28_days,
   'Modelo': 'XGBOOST'
    })

# Almacenar las predicciones en la lista
predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_por_total.txt')

Tienda XGB: Total Tiendas


### PREDICCIONES POR TIENDA

In [3]:
ventas_tienda = df_venart.groupby([df_venart['fecha'].dt.date,'store'])['cifra'].sum().unstack().fillna(0).reset_index()
ventas_tienda.columns.name = None

predicciones_TOTALES = pd.DataFrame()

rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_tienda['fecha'] = pd.to_datetime(ventas_tienda['fecha'], format = '%Y-%m-%d')
ventas_tienda = pd.merge(pd.DataFrame({'fecha': rango}), ventas_tienda, on='fecha', how='left')
ventas_tienda = ventas_tienda.fillna(0)
ventas_tienda.set_index('fecha', inplace=True)
ventas_tienda['Total Tiendas'] = ventas_tienda.iloc[:, 0:].sum(axis=1)
ventas_tienda['evento'] = ventas_tienda.index.isin(df_eventos['date']) 

for i, tienda in enumerate(tiendas_XGB):
        print(f"{i} Tienda XGB: {tienda}")
        df = ventas_tienda[[tienda, 'evento']].copy()
        if tienda in tiendas_XGB_24_L30:
            df = df.loc['2014-04-24':'2016-04-24']
        # Agregar más características relevantes
        df['mes'] = df.index.month
        df['dia_semana'] = df.index.weekday
    
        # Agregar los lags de 7, 30 y 365 días
        df['lag_1'] = df[tienda].shift(1)
        df['lag_7'] = df[tienda].shift(7)
        df['lag_30'] = df[tienda].shift(30)
        if tienda in tiendas_XGB_SC_L365:
            df['lag_365'] = df[tienda].shift(365)
  
        # Eliminar filas con valores nulos que surgen por los lags
        df = df.dropna()

        # Preparar los datos para XGBoost, incluyendo las nuevas características de lag
        if tienda in tiendas_XGB_SC_L365:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
        
        else:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         
        y_train = df[tienda].values

        ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
        md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
        model.fit(X_train, y_train)

        # Predecir los próximos 28 días después del conjunto de prueba
        future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
        future_mes = future_dates.month
        future_dia_semana = future_dates.weekday
    
        # Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
        future_lag_1 = df[tienda].shift(1).iloc[-28:].values
        future_lag_7 = df[tienda].shift(7).iloc[-28:].values
        future_lag_30 = df[tienda].shift(30).iloc[-28:].values
        if tienda in tiendas_XGB_SC_L365:
            future_lag_365 = df[tienda].shift(365).iloc[-28:].values
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
        else:
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

        forecast_28_days = model.predict(X_future)

        # Crear DataFrame con las predicciones
        df_predicciones = pd.DataFrame({
            'Fecha': future_dates,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Modelo': 'XGBOOST'
      })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

for i, tienda in enumerate(tiendas_SARIMAX):  
        print(f"{i} Tienda ARM: {tienda}")
        df = ventas_tienda[[tienda, 'evento']].copy()
        df['evento'] = df['evento'].astype(int)

        # ENTRENO EL MODELO CON TODOS LOS DATOS
        sarimax_model = SARIMAX(df[tienda], exog=df[['evento']],
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 7),
                            initialization='approximate_diffuse')
        results_sarimax = sarimax_model.fit(disp=False)

        fechas_futuras = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        forecast_28_days = results_sarimax.predict(start=len(df), end=len(df) + 27, exog=pd.DataFrame({'evento': [0] * 28}))

         # Crear un DataFrame para almacenar las predicciones de cada tienda
        df_predicciones = pd.DataFrame({
            'Fecha': fechas_futuras,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Modelo': 'SARIMAX'
            })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_por_tienda.txt')

0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury


### PREDICCIONES A NIVEL DE CATEGORIA

In [19]:

ventas_tienda_cat = df_venart.groupby([df_venart['fecha'].dt.date, 'category','store'])['cifra'].sum().unstack().fillna(0).reset_index()
ventas_tienda_cat.columns.name = None

predicciones_TOTALES = pd.DataFrame()

for categoria in ventas_tienda_cat['category'].unique():
    print(f"cat: {categoria}")
    ventas_cat = ventas_tienda_cat.loc[ventas_tienda_cat['category'] == categoria]
    ventas_cat = ventas_cat.drop(columns=['category'])
    ventas_cat['fecha'] = pd.to_datetime(ventas_cat['fecha'])
    rango = pd.date_range(start="2011-01-29", end="2016-04-24")
    ventas_cat = pd.merge(pd.DataFrame({'fecha': rango}), ventas_cat, on='fecha', how='left')
    ventas_cat = ventas_cat.fillna(0)
    ventas_cat.set_index('fecha', inplace=True)
    ventas_cat['Total Tiendas'] = ventas_cat.iloc[:, 0:].sum(axis=1)
    ventas_cat['evento'] = ventas_cat.index.isin(df_eventos['date']) 

    for i, tienda in enumerate(tiendas_XGB):
        print(f"{i} Tienda XGB: {tienda}")
        df = ventas_cat[[tienda, 'evento']].copy()
        if tienda in tiendas_XGB_24_L30:
            df = df.loc['2014-04-24':'2016-04-24']
        # Agregar más características relevantes
        df['mes'] = df.index.month
        df['dia_semana'] = df.index.weekday
    
        # Agregar los lags de 7, 30 y 365 días
        df['lag_1'] = df[tienda].shift(1)
        df['lag_7'] = df[tienda].shift(7)
        df['lag_30'] = df[tienda].shift(30)
        if tienda in tiendas_XGB_SC_L365:
            df['lag_365'] = df[tienda].shift(365)
  
        # Eliminar filas con valores nulos que surgen por los lags
        df = df.dropna()

        # Preparar los datos para XGBoost, incluyendo las nuevas características de lag
        if tienda in tiendas_XGB_SC_L365:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
        
        else:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         
        y_train = df[tienda].values

        ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
        md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
        model.fit(X_train, y_train)

        # Predecir los próximos 28 días después del conjunto de prueba
        future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
        future_mes = future_dates.month
        future_dia_semana = future_dates.weekday
    
        # Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
        future_lag_1 = df[tienda].shift(1).iloc[-28:].values
        future_lag_7 = df[tienda].shift(7).iloc[-28:].values
        future_lag_30 = df[tienda].shift(30).iloc[-28:].values
        if tienda in tiendas_XGB_SC_L365:
            future_lag_365 = df[tienda].shift(365).iloc[-28:].values
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
        else:
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

        forecast_28_days = model.predict(X_future)

        # Crear DataFrame con las predicciones
        df_predicciones = pd.DataFrame({
            'Fecha': future_dates,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'categoria': categoria,
            'Modelo': 'XGBOOST'
      })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

    for i, tienda in enumerate(tiendas_SARIMAX):  
        print(f"{i} Tienda ARM: {tienda}")
        df = ventas_cat[[tienda, 'evento']].copy()
        df['evento'] = df['evento'].astype(int)

        # ENTRENO EL MODELO CON TODOS LOS DATOS
        sarimax_model = SARIMAX(df[tienda], exog=df[['evento']],
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 7),
                            initialization='approximate_diffuse')
        results_sarimax = sarimax_model.fit(disp=False)

        fechas_futuras = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        forecast_28_days = results_sarimax.predict(start=len(df), end=len(df) + 27, exog=pd.DataFrame({'evento': [0] * 28}))

         # Crear un DataFrame para almacenar las predicciones de cada tienda
        df_predicciones = pd.DataFrame({
            'Fecha': fechas_futuras,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'categoria': categoria,
            'Modelo': 'SARIMAX'
            })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_por_cat.txt')

cat: ACCESORIES
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
cat: HOME_&_GARDEN
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
cat: SUPERMARKET
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury


### PREDICCIONES A NIVEL DE DEPARTAMENTO

In [20]:
ventas_tienda_dpto = df_venart.groupby([df_venart['fecha'].dt.date, 'department','store'])['cifra'].sum().unstack().fillna(0).reset_index()
ventas_tienda_dpto.columns.name = None

predicciones_TOTALES = pd.DataFrame()

for departamento in ventas_tienda_dpto['department'].unique():
    print(f"Dpto: {departamento}")
    ventas_dpto = ventas_tienda_dpto.loc[ventas_tienda_dpto['department'] == departamento]
    ventas_dpto = ventas_dpto.drop(columns=['department'])
    ventas_dpto['fecha'] = pd.to_datetime(ventas_dpto['fecha'])
    rango = pd.date_range(start="2011-01-29", end="2016-04-24")
    ventas_dpto = pd.merge(pd.DataFrame({'fecha': rango}), ventas_dpto, on='fecha', how='left')
    ventas_dpto = ventas_dpto.fillna(0)
    ventas_dpto.set_index('fecha', inplace=True)
    ventas_dpto['Total Tiendas'] = ventas_dpto.iloc[:, 0:].sum(axis=1)
    ventas_dpto['evento'] = ventas_dpto.index.isin(df_eventos['date']) 

    for i, tienda in enumerate(tiendas_XGB):
        print(f"{i} Tienda XGB: {tienda}")
        df = ventas_dpto[[tienda, 'evento']].copy()
        if tienda in tiendas_XGB_24_L30:
            df = df.loc['2014-04-24':'2016-04-24']
        # Agregar más características relevantes
        df['mes'] = df.index.month
        df['dia_semana'] = df.index.weekday
    
        # Agregar los lags de 7, 30 y 365 días
        df['lag_1'] = df[tienda].shift(1)
        df['lag_7'] = df[tienda].shift(7)
        df['lag_30'] = df[tienda].shift(30)
        if tienda in tiendas_XGB_SC_L365:
            df['lag_365'] = df[tienda].shift(365)
  
        # Eliminar filas con valores nulos que surgen por los lags
        df = df.dropna()

        # Preparar los datos para XGBoost, incluyendo las nuevas características de lag
        if tienda in tiendas_XGB_SC_L365:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
        
        else:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         
        y_train = df[tienda].values

        ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
        md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
        model.fit(X_train, y_train)

        # Predecir los próximos 28 días después del conjunto de prueba
        future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
        future_mes = future_dates.month
        future_dia_semana = future_dates.weekday
    
        # Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
        future_lag_1 = df[tienda].shift(1).iloc[-28:].values
        future_lag_7 = df[tienda].shift(7).iloc[-28:].values
        future_lag_30 = df[tienda].shift(30).iloc[-28:].values
        if tienda in tiendas_XGB_SC_L365:
            future_lag_365 = df[tienda].shift(365).iloc[-28:].values
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
        else:
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

        forecast_28_days = model.predict(X_future)

        # Crear DataFrame con las predicciones
        df_predicciones = pd.DataFrame({
            'Fecha': future_dates,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Departamento': departamento,
            'Modelo': 'XGBOOST'
      })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

    for i, tienda in enumerate(tiendas_SARIMAX):  
        print(f"{i} Tienda ARM: {tienda}")
        df = ventas_dpto[[tienda, 'evento']].copy()
        df['evento'] = df['evento'].astype(int)

        # ENTRENO EL MODELO CON TODOS LOS DATOS
        sarimax_model = SARIMAX(df[tienda], exog=df[['evento']],
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 7),
                            initialization='approximate_diffuse')
        results_sarimax = sarimax_model.fit(disp=False)

        fechas_futuras = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        forecast_28_days = results_sarimax.predict(start=len(df), end=len(df) + 27, exog=pd.DataFrame({'evento': [0] * 28}))

         # Crear un DataFrame para almacenar las predicciones de cada tienda
        df_predicciones = pd.DataFrame({
            'Fecha': fechas_futuras,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Departamento': departamento,
            'Modelo': 'SARIMAX'
            })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_por_dpto.txt')

Dpto: ACCESORIES_1
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
Dpto: ACCESORIES_2
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
Dpto: HOME_&_GARDEN_1
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
Dpto: HOME_&_GARDEN_2
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tie

### PREDICCIONES A NIVEL DE ARTÍCULO

In [27]:
ventas_tienda_art = df_venart.groupby([df_venart['fecha'].dt.date, 'item','store'])['cifra'].sum().unstack().fillna(0).reset_index()
ventas_tienda_art.columns.name = None

predicciones_TOTALES = pd.DataFrame()
j = 0
for articulo in sorted(ventas_tienda_art['item'].unique()):
    j += 1
    print(f"art: {articulo} -> {j} de 3049 -> {j/3049}")
    ventas_art = ventas_tienda_art.loc[ventas_tienda_art['item'] == articulo]
    ventas_art = ventas_art.drop(columns=['item'])
    ventas_art['fecha'] = pd.to_datetime(ventas_art['fecha'])
    rango = pd.date_range(start="2011-01-29", end="2016-04-24")
    ventas_art = pd.merge(pd.DataFrame({'fecha': rango}), ventas_art, on='fecha', how='left')
    ventas_art = ventas_art.fillna(0)
    ventas_art.set_index('fecha', inplace=True)
    ventas_art['Total Tiendas'] = ventas_art.iloc[:, 0:].sum(axis=1)
    ventas_art['evento'] = ventas_art.index.isin(df_eventos['date']) 

    for i, tienda in enumerate(tiendas_XGB):
        print(f"{i} Tienda XGB: {tienda}")
        df = ventas_art[[tienda, 'evento']].copy()
        if tienda in tiendas_XGB_24_L30:
            df = df.loc['2014-04-24':'2016-04-24']
        # Agregar más características relevantes
        df['mes'] = df.index.month
        df['dia_semana'] = df.index.weekday
    
        # Agregar los lags de 7, 30 y 365 días
        df['lag_1'] = df[tienda].shift(1)
        df['lag_7'] = df[tienda].shift(7)
        df['lag_30'] = df[tienda].shift(30)
        if tienda in tiendas_XGB_SC_L365:
            df['lag_365'] = df[tienda].shift(365)
  
        # Eliminar filas con valores nulos que surgen por los lags
        df = df.dropna()

        # Preparar los datos para XGBoost, incluyendo las nuevas características de lag
        if tienda in tiendas_XGB_SC_L365:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30', 'lag_365']].values
        
        else:
            X_train = df[['evento', 'mes', 'dia_semana', 'lag_1','lag_7', 'lag_30']].values
         

        y_train = df[tienda].values
     

        ne = df_hp.loc[df_hp['tienda'] == tienda, 'n_estimators'].values[0]
        md = df_hp.loc[df_hp['tienda'] == tienda, 'max_depth'].values[0]
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=ne,learning_rate=0.1, max_depth=md)
        model.fit(X_train, y_train)

        # Predecir los próximos 28 días después del conjunto de prueba
        future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        future_eventos = pd.Series(future_dates).apply(lambda x: x in df_eventos['date'].values)
        future_mes = future_dates.month
        future_dia_semana = future_dates.weekday
    
        # Predecir los futuros 28 días (para las lags futuras se utiliza el forecast de días anteriores)
        future_lag_1 = df[tienda].shift(1).iloc[-28:].values
        future_lag_7 = df[tienda].shift(7).iloc[-28:].values
        future_lag_30 = df[tienda].shift(30).iloc[-28:].values
        if tienda in tiendas_XGB_SC_L365:
            future_lag_365 = df[tienda].shift(365).iloc[-28:].values
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30, future_lag_365))
        else:
            X_future = np.column_stack((future_eventos, future_mes, future_dia_semana, future_lag_1, future_lag_7, future_lag_30))

        forecast_28_days = model.predict(X_future)

        # Crear DataFrame con las predicciones
        df_predicciones = pd.DataFrame({
            'Fecha': future_dates,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Articulo': articulo,
            'Modelo': 'XGBOOST'
      })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

    for i, tienda in enumerate(tiendas_SARIMAX):  
        print(f"{i} Tienda ARM: {tienda}")
        df = ventas_art[[tienda, 'evento']].copy()
        df['evento'] = df['evento'].astype(int)

        # ENTRENO EL MODELO CON TODOS LOS DATOS
        sarimax_model = SARIMAX(df[tienda], exog=df[['evento']],
                            order=(1, 1, 1),
                            seasonal_order=(1, 1, 1, 7),
                            initialization='approximate_diffuse')
        results_sarimax = sarimax_model.fit(disp=False)

        fechas_futuras = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=28)
        forecast_28_days = results_sarimax.predict(start=len(df), end=len(df) + 27, exog=pd.DataFrame({'evento': [0] * 28}))

         # Crear un DataFrame para almacenar las predicciones de cada tienda
        df_predicciones = pd.DataFrame({
            'Fecha': fechas_futuras,
            'Prediccion_Ventas': forecast_28_days,
            'Tienda': tienda,
            'Articulo': articulo,
            'Modelo': 'SARIMAX'
            })

        # Almacenar las predicciones en la lista
        predicciones_TOTALES = pd.concat([predicciones_TOTALES, df_predicciones], ignore_index=True)

predicciones_TOTALES.to_csv('Pred_por_art.txt')

art: ACCESORIES_1_001 -> 1 de 3049 -> 0.00032797638570022957
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
art: ACCESORIES_1_002 -> 2 de 3049 -> 0.0006559527714004591
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM: Roxbury
art: ACCESORIES_1_003 -> 3 de 3049 -> 0.0009839291571006887
0 Tienda XGB: Back_Bay
1 Tienda XGB: Yorktown
2 Tienda XGB: Harlem
3 Tienda XGB: Total Tiendas
4 Tienda XGB: Greenwich_Village
5 Tienda XGB: Queen_Village
6 Tienda XGB: South_End
7 Tienda XGB: Tribeca
0 Tienda ARM: Brooklyn
1 Tienda ARM: Midtown_Village
2 Tienda ARM:

In [11]:
ventas_tienda = df_venart.groupby([df_venart['fecha'].dt.date,'store'])['cifra'].sum().unstack().fillna(0).reset_index()
ventas_tienda.columns.name = None
rango = pd.date_range(start="2011-01-29", end="2016-04-24")
ventas_tienda['fecha'] = pd.to_datetime(ventas_tienda['fecha'], format = '%Y-%m-%d')
ventas_tienda = pd.merge(pd.DataFrame({'fecha': rango}), ventas_tienda, on='fecha', how='left')
ventas_tienda = ventas_tienda.fillna(0)
ventas_tienda.set_index('fecha', inplace=True)
ventas_tienda['Total Tiendas'] = ventas_tienda.iloc[:, 0:].sum(axis=1)
ventas_tienda['evento'] = ventas_tienda.index.isin(df_eventos['date']) 

In [13]:
ventas_tienda

,Back_Bay,Brooklyn,Greenwich_Village,Harlem,Midtown_Village,Queen_Village,Roxbury,South_End,Tribeca,Yorktown,Total Tiendas,evento
fecha,,,,,,,,,,,,
2011-01-29,9331.27,5604.83,13413.55,11222.22,8019.00,11492.99,12203.88,8097.15,14336.85,6716.58,100438.32,False
2011-01-30,9028.83,5737.84,11971.81,10385.80,7021.16,12099.23,12070.56,8128.08,14916.24,5706.22,97065.77,False
2011-01-31,6632.56,4472.59,8847.92,6578.61,4517.85,9242.54,8181.91,5578.07,11203.18,5672.54,70927.77,False
2011-02-01,6866.44,4565.62,9074.16,6871.80,3646.74,8779.90,8567.87,6575.04,12576.93,7047.48,74571.98,False
2011-02-02,4980.22,4709.20,8043.59,6467.75,3.55,5649.41,7428.37,5309.10,11726.32,3268.67,57586.18,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,13721.02,9855.64,15223.97,14237.75,12221.15,11450.71,13949.81,11040.97,22388.94,14961.69,139051.65,False
2016-04-21,14282.03,10093.89,14404.03,13218.05,12764.61,11321.75,14105.43,10169.71,20012.54,15658.31,136030.35,False
2016-04-22,14719.82,10213.23,17514.33,18044.06,15153.46,13464.63,15456.44,11728.10,22835.76,17459.97,156589.80,False


In [14]:
predicciones_tienda = predicciones_TOTALES.groupby([predicciones_TOTALES['Fecha'].dt.date,'Tienda'])['Prediccion_Ventas'].sum().unstack().fillna(0).reset_index()
predicciones_tienda.columns.name = None
#rango = pd.date_range(start="2011-01-29", end="2016-04-24")
predicciones_tienda['Fecha'] = pd.to_datetime(predicciones_tienda['Fecha'], format = '%Y-%m-%d')
#predicciones_tienda = pd.merge(pd.DataFrame({'fecha': rango}), predicciones_tienda, on='fecha', how='left')
predicciones_tienda = predicciones_tienda.fillna(0)
predicciones_tienda.set_index('Fecha', inplace=True)
#predicciones_tienda['Total Tiendas'] = predicciones_tienda.iloc[:, 0:].sum(axis=1)
predicciones_tienda['evento'] = predicciones_tienda.index.isin(df_eventos['date']) 

In [15]:
predicciones_tienda

,Back_Bay,Brooklyn,Greenwich_Village,Harlem,Midtown_Village,Queen_Village,Roxbury,South_End,Total Tiendas,Tribeca,Yorktown,evento
Fecha,,,,,,,,,,,,
2016-04-25,14293.57,11413.48,16231.52,14650.70,12070.68,10686.30,15350.82,11541.33,143419.28,23890.62,14365.17,False
2016-04-26,13943.57,10468.57,14910.30,13849.99,11444.26,10245.00,13960.55,11344.54,136495.97,21925.85,15350.78,False
2016-04-27,13042.25,10050.88,14788.51,13801.49,11789.45,9862.05,14034.74,10726.85,131923.33,21822.14,14908.19,False
2016-04-28,14061.88,10152.48,15206.82,13999.20,12290.06,9732.87,14312.63,11077.56,128369.13,21915.57,14523.25,False
2016-04-29,16920.44,10323.95,19268.88,17783.09,15078.54,14025.42,15278.77,12221.01,160928.88,23699.70,18642.28,False
2016-04-30,18068.13,11932.98,23217.86,23239.56,19500.65,17063.08,18358.69,15422.96,198964.61,29988.61,23074.17,False
2016-05-01,19219.78,12545.14,22941.04,24081.56,18431.11,16614.96,19294.62,16511.08,190247.98,32894.91,19742.30,False
2016-05-02,16575.88,11239.53,18012.49,14650.70,12155.76,13582.53,15609.10,11940.37,150592.36,27083.39,17175.15,False
2016-05-03,14854.03,10441.13,16024.51,15331.59,11503.49,12625.19,13908.71,12392.65,142794.67,23886.09,16625.38,False


In [17]:
# VISUALIZACIÓN DE PREDICCIONES
import plotly.graph_objects as go
for tienda in predicciones_tienda.columns:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ventas_tienda.index, y=ventas_tienda[tienda], mode='lines', name='Datos históricos'))
    fig.add_trace(go.Scatter(x=predicciones_tienda.index, y=predicciones_tienda[tienda], mode='lines', name='Predicciones'))      
    

    # AÑADO LAS MARCAS DE LOS DÍAS FESTIVOS POR SI AYUDAN A INTERPORETAR EL MODELO
    for _, evento in df_eventos.iterrows():
        evento_date = evento['date']
        evento_name = evento['event']
        fig.add_vline(x=evento_date, line=dict(color="red", width=2, dash="dash"))
        fig.add_annotation(x=evento_date, y=max(ventas_tienda[tienda].max(), max(predicciones_tienda[tienda])), text=evento_name, showarrow=True, arrowhead=1, textangle=-90, valign="top")
        
    
    # ALGUNOS AJUSTES DE DISEÑO DE LOS GRÁFICOS
    fig.update_layout(
        title= f'{tienda} Predicción Final',
        xaxis_title='Fecha',
        yaxis_title=tienda,
        legend=dict(x=0.4, y=1.4),
        xaxis=dict(
            tickformat='%Y-%m-%d',
            tickmode='auto',
            nticks=100,  # Ajusta el número de etiquetas en el eje x
            range=['2014-04-25', '2016-05-23']  # Limita el rango de fechas
        ),
        template='plotly_white'
    )

    fig.show()